<a href="https://colab.research.google.com/github/hyeonu96/first-repository/blob/main/%5BEX_10%5D%ED%94%84%EB%A1%9C%EC%A0%9D%ED%8A%B8_%EB%8B%A8%EC%96%B4_Level%EB%A1%9C_%EB%B2%88%EC%97%AD%EA%B8%B0_%EC%97%85%EA%B7%B8%EB%A0%88%EC%9D%B4%EB%93%9C%ED%95%98%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 프로젝트 : 단어 Level로 번역기 업그레이드하기

In [1]:
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import numpy as np
import re    
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Masking, Dropout
from tensorflow.keras.models import Model

In [2]:
import os
file_path = '/content/drive/MyDrive/fra/fra.txt'
lines = pd.read_csv(file_path, names=['eng', 'fra', 'cc'], sep='\t')
print('전체 샘플의 수:', len(lines))
lines.sample(5)

전체 샘플의 수: 197463


,eng,fra,cc
3013,He's asleep.,Il est endormi.,CC-BY 2.0 (France) Attribution: tatoeba.org #1...
66347,She's unfit for the job.,Elle est inapte pour le poste.,CC-BY 2.0 (France) Attribution: tatoeba.org #1...
26813,Tom never arrived.,Tom n'est jamais arrivé.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
104888,Practice is the best teacher.,La pratique est le meilleur maître.,CC-BY 2.0 (France) Attribution: tatoeba.org #6...
74435,That's not very romantic.,Ce n'est pas très romantique.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...


In [3]:
lines = lines[['eng', 'fra']][60000:93000]
lines.sample(5)

,eng,fra
73253,It was just a hypothesis.,Ce n'était qu'une hypothèse.
67101,They stopped doing that.,Ils ont arrêté de faire ça.
64100,I hope you're listening.,J'espère que tu écoutes.
87817,I need to talk to a lawyer.,Je dois parler à un avocat.
92649,You should act more calmly.,Tu devrais te comporter plus calmement.


In [4]:
lines_np_eng= lines['eng'].to_numpy()
lines_np_fra= lines['fra'].to_numpy()
lines_np_eng

array(['Those are your choices.', 'Those are your options.',
       'Those are your options.', ..., 'Are you coming to the party?',
       'Are you coming to the party?', 'Are you done with the paper?'],
      dtype=object)

## 데이터 전처리

### 소문자 변경 후 구두점 분리

In [5]:
sos_token = '<start> '
eos_token = ' <end>'

def preprocess_line(line, plus_token = True):
    # 소문자로 변경하기
    line = line.lower().strip()
    # 구두점(Punctuation)을 단어와 분리하기
    line = re.sub(r"([?.!,¿])", r" \1 ", line)
    line = re.sub(r'[" "]+', " ", line)
    line = re.sub(r"[^a-zA-Z?.!,¿]+", " ", line)

    line = line.strip()
    
    if plus_token == True:
        line = sos_token + line + eos_token
    
    return line

### 띄어쓰기 단위로 토큰화 함수

In [6]:
def tokenize(corpus):
    tokenizer = Tokenizer(
        num_words=7000,  
        filters=' ',   
        oov_token="<unk>"  
    )
    tokenizer.fit_on_texts(corpus)  

    tensor = tokenizer.texts_to_sequences(corpus)   

    return tensor, tokenizer

### 영어, 프랑스어 전처리하기

In [7]:
eng_lines = []
fra_lines = []

# eng_lines.append(lines.eng.apply(lambda x : preprocess_line(x,plus_token = False)))
# fra_lines.append(lines.fra.apply(lambda x : preprocess_line(x),))

for eng, fra in zip(lines.eng, lines.fra):
    if len(eng) == 0: continue
    if len(fra) == 0: continue   
        
    eng_lines.append(preprocess_line(eng, plus_token = False))
    fra_lines.append(preprocess_line(fra))

In [8]:
np.shape(eng_lines)

(33000,)

In [9]:
eng_tensor, eng_tokenizer = tokenize(eng_lines)
fra_tensor, fra_tokenizer = tokenize(fra_lines)
fra_tensor[:10]

[[2, 1155, 175, 1274, 4, 3],
 [2, 1064, 22, 61, 42, 1274, 4, 3],
 [2, 21, 61, 175, 1274, 4, 3],
 [2, 21, 61, 42, 1274, 4, 3],
 [2, 178, 672, 36, 480, 98, 4, 3],
 [2, 178, 316, 27, 14, 61, 9, 44, 4, 3],
 [2, 33, 59, 30, 126, 2309, 378, 4, 3],
 [2, 178, 897, 61, 2646, 4, 3],
 [2, 17, 89, 8, 24, 3799, 316, 4, 3],
 [2, 33, 8, 499, 47, 44, 4, 3]]

### input, target 설정

In [10]:
encoder_input = eng_tensor
# 종료 토큰 제거
decoder_input = [[char for char in line if char != fra_tokenizer.word_index['<end>']] for line in fra_tensor]
# 시작 토큰 제거
decoder_target =[[char for char in line if char != fra_tokenizer.word_index['<start>']] for line in fra_tensor]

### padding 추가

In [11]:
def pad_tensor(tensor):
    total_data_text = list(tensor)
    num_tokens = [len(tokens) for tokens in total_data_text]
    max_tokens = max(num_tokens)
#     max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
    maxlen = int(max_tokens)
    tensor = pad_sequences(tensor, padding='post', maxlen=maxlen)  
    return tensor

In [12]:
encoder_input = pad_tensor(encoder_input)
decoder_input = pad_tensor(decoder_input)
decoder_target = pad_tensor(decoder_target)
print('영어 데이터의 크기(shape) :',np.shape(encoder_input))
print('프랑스어 입력데이터의 크기(shape) :',np.shape(decoder_input))
print('프랑스어 출력데이터의 크기(shape) :',np.shape(decoder_target))

영어 데이터의 크기(shape) : (33000, 10)
프랑스어 입력데이터의 크기(shape) : (33000, 17)
프랑스어 출력데이터의 크기(shape) : (33000, 17)


In [13]:
eng_vocab_size = len(eng_tokenizer.word_index)+1
fra_vocab_size = len(fra_tokenizer.word_index)+1

max_eng_seq_len = encoder_input.shape[1]
max_fra_seq_len = decoder_input.shape[1]

In [14]:
print('전체 샘플의 수 :',len(lines))
print('영어 단어장의 크기 :', eng_vocab_size)
print('프랑스어 단어장의 크기 :', fra_vocab_size)
print('영어 시퀀스의 최대 길이', max_eng_seq_len)
print('프랑스어 시퀀스의 최대 길이', max_fra_seq_len)

전체 샘플의 수 : 33000
영어 단어장의 크기 : 5975
프랑스어 단어장의 크기 : 8501
영어 시퀀스의 최대 길이 10
프랑스어 시퀀스의 최대 길이 17


### train, test dataset 나누기

In [15]:
indices = np.arange(encoder_input.shape[0])
np.random.shuffle(indices)

encoder_input = encoder_input[indices]
decoder_input = decoder_input[indices]
decoder_target = decoder_target[indices]

In [16]:
n_of_val = 3000

encoder_input_train = encoder_input[:-n_of_val]
decoder_input_train = decoder_input[:-n_of_val]
decoder_target_train = decoder_target[:-n_of_val]

encoder_input_test = encoder_input[-n_of_val:]
decoder_input_test = decoder_input[-n_of_val:]
decoder_target_test = decoder_target[-n_of_val:]

print(encoder_input_train.shape)
print(decoder_input_train.shape)
print(decoder_target_train.shape)
print(encoder_input_test.shape)
print(decoder_input_test.shape)
print(decoder_target_test.shape)

(30000, 10)
(30000, 17)
(30000, 17)
(3000, 10)
(3000, 17)
(3000, 17)


## 임베딩 층(Embedding layer) 사용하기

### 인코더

In [17]:
embedding_size = 512
hidden_size = 512
# 인코더에서 사용할 임베딩 층 사용 예시
encoder_inputs = Input(shape=(None, ), name='encoder_input')
enc_emb =  Embedding(eng_vocab_size, embedding_size,
                    input_length=max_eng_seq_len)(encoder_inputs)
enc_masking = Masking(mask_value=0.0)(enc_emb)
encoder_lstm = LSTM(hidden_size, dropout = 0.5, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_masking)
encoder_states = [state_h, state_c]

### 디코더

In [18]:
decoder_inputs = Input(shape=(None, ), name='decoder_input')
dec_emb =  Embedding(fra_vocab_size, embedding_size)(decoder_inputs)
dec_masking = Masking(mask_value=0.0)(dec_emb)
decoder_lstm = LSTM(hidden_size, dropout = 0.5, return_sequences = True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_masking, initial_state = encoder_states)

In [19]:
decoder_softmax_layer = Dense(fra_vocab_size, activation='softmax')
decoder_outputs = decoder_softmax_layer(decoder_outputs)

In [20]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy')

In [21]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 encoder_input (InputLayer)     [(None, None)]       0           []                               
                                                                                                  
 decoder_input (InputLayer)     [(None, None)]       0           []                               
                                                                                                  
 embedding (Embedding)          (None, None, 512)    3059200     ['encoder_input[0][0]']          
                                                                                                  
 embedding_1 (Embedding)        (None, None, 512)    4352512     ['decoder_input[0][0]']          
                                                                                              

In [22]:
model.fit(x=[encoder_input_train, decoder_input_train], 
          y=decoder_target_train, 
          validation_data = ([encoder_input_test, decoder_input_test], 
                             decoder_target_test),
          batch_size=32, 
          epochs=50)

Epoch 1/50
938/938 [==============================] - 33s 22ms/step - loss: 2.0065 - val_loss: 1.6640
Epoch 2/50
938/938 [==============================] - 18s 19ms/step - loss: 1.5139 - val_loss: 1.4295
Epoch 3/50
938/938 [==============================] - 18s 19ms/step - loss: 1.3135 - val_loss: 1.2842
Epoch 4/50
938/938 [==============================] - 18s 19ms/step - loss: 1.1745 - val_loss: 1.1958
Epoch 5/50
938/938 [==============================] - 18s 19ms/step - loss: 1.0676 - val_loss: 1.1341
Epoch 6/50
938/938 [==============================] - 18s 19ms/step - loss: 0.9853 - val_loss: 1.0896
Epoch 7/50
938/938 [==============================] - 18s 19ms/step - loss: 0.9179 - val_loss: 1.0604
Epoch 8/50
938/938 [==============================] - 18s 19ms/step - loss: 0.8638 - val_loss: 1.0412
Epoch 9/50
938/938 [==============================] - 18s 19ms/step - loss: 0.8217 - val_loss: 1.0312
Epoch 10/50
938/938 [==============================] - 18s 19ms/step - loss: 0.785

## 모델 구현하기

### 인코더

In [23]:
encoder_model = Model(inputs = encoder_inputs, outputs = encoder_states)
encoder_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder_input (InputLayer)  [(None, None)]            0         
                                                                 
 embedding (Embedding)       (None, None, 512)         3059200   
                                                                 
 masking (Masking)           (None, None, 512)         0         
                                                                 
 lstm (LSTM)                 [(None, 512),             2099200   
                              (None, 512),                       
                              (None, 512)]                       
                                                                 
Total params: 5,158,400
Trainable params: 5,158,400
Non-trainable params: 0
_________________________________________________________________


### 디코더

In [24]:
decoder_state_input_h = Input(shape=(embedding_size,))
decoder_state_input_c = Input(shape=(embedding_size,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2 = Embedding(fra_vocab_size, embedding_size)(decoder_inputs)
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state = decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]

decoder_outputs2 = decoder_softmax_layer(decoder_outputs2)

In [25]:
eng2idx = eng_tokenizer.word_index
fra2idx = fra_tokenizer.word_index
idx2eng = eng_tokenizer.index_word
idx2fra = fra_tokenizer.index_word

In [26]:
decoder_model = Model(inputs=[decoder_inputs] + decoder_states_inputs, outputs=[decoder_outputs2] + decoder_states2)
decoder_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 decoder_input (InputLayer)     [(None, None)]       0           []                               
                                                                                                  
 embedding_2 (Embedding)        (None, None, 512)    4352512     ['decoder_input[0][0]']          
                                                                                                  
 input_1 (InputLayer)           [(None, 512)]        0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 512)]        0           []                               
                                                                                            

In [27]:
def decode_sequence(input_seq):
    # 입력으로부터 인코더의 상태를 얻음
    states_value = encoder_model.predict(input_seq)

    # <start>에 해당하는 원-핫 벡터 생성
    target_seq = np.zeros((1,1))
    target_seq[0, 0] = fra2idx['<start>']
    
    stop_condition = False
    decoded_sentence = ""

    # stop_condition이 True가 될 때까지 루프 반복
    while not stop_condition:
        # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # 예측 결과를 문자로 변환
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = idx2fra[sampled_token_index]

        # 현재 시점의 예측 문자를 예측 문장에 추가
        decoded_sentence += ' '+sampled_char

        # <eos>에 도달하거나 최대 길이를 넘으면 중단.
        if (sampled_char == '<end>' or
           len(decoded_sentence) > max_fra_seq_len):
            stop_condition = True

        # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index

        # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
        states_value = [h, c]

    return decoded_sentence

In [28]:
# 원문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq2src(input_seq):
    temp=''
    for i in input_seq:
        if(i!=0):
            temp = temp + idx2eng[i]+' '
    return temp

In [29]:
# 번역문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq2tar(input_seq):
    temp=''
    for i in input_seq:
        if((i!=0 and i!=fra2idx['<start>']) and i!=fra2idx['<end>']):
            temp = temp + idx2fra[i] + ' '
    return temp

## 모델 평가하기

In [30]:
for seq_index in [1,201,501,1004,2015]:
    input_seq = encoder_input_test[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print(35 * "-")
    print('입력 문장:', seq2src(encoder_input_test[seq_index]))
    print('정답 문장:', seq2tar(decoder_input_test[seq_index]))
    print('번역기가 번역한 문장:', decoded_sentence[:len(decoded_sentence)-1])

1/1 [==============================] - 0s 16ms/step
-----------------------------------
입력 문장: i d like to take a walk . 
정답 문장: j aimerais faire une promenade . 
번역기가 번역한 문장:  je voudrais fair
1/1 [==============================] - 0s 15ms/step
-----------------------------------
입력 문장: who was here this morning ? 
정답 문장: qui tait l ce matin ? 
번역기가 번역한 문장:  qui tait l l l l 
1/1 [==============================] - 0s 15ms/step
-----------------------------------
입력 문장: that s all i can ask for . 
정답 문장: c est tout ce que je peux demander . 
번역기가 번역한 문장:  c me c je je pou
1/1 [==============================] - 0s 14ms/step
-----------------------------------
입력 문장: your room number , please . 
정답 문장: voici votre num ro de chambre . 
번역기가 번역한 문장:  ta ta votre ta l 
1/1 [==============================] - 0s 15ms/step
-----------------------------------
입력 문장: i m tired of doing this . 
정답 문장: je suis fatigu de faire ceci . 
번역기가 번역한 문장:  je en d d faire fair


회고

-----------------------------------
- 입력 문장: i d like to take a walk . 
- 정답 문장: j aimerais faire une promenade . 
- 번역기가 번역한 문장:  je voudrais fair
-----------------------------------
- 입력 문장: who was here this morning ? 
- 정답 문장: qui tait l ce matin ? 
- 번역기가 번역한 문장:  qui tait l l l l 
-----------------------------------
- 입력 문장: that s all i can ask for . 
- 정답 문장: c est tout ce que je peux demander . 
- 번역기가 번역한 문장:  c me c je je pou
-----------------------------------
- 입력 문장: your room number , please . 
- 정답 문장: voici votre num ro de chambre . 
- 번역기가 번역한 문장:  ta ta votre ta l 
-----------------------------------
- 입력 문장: i m tired of doing this . 
- 정답 문장: je suis fatigu de faire ceci . 
- 번역기가 번역한 문장:  je en d d faire fair
-----------------------------------

모델을 통해 뽑아낸 결과값들이다. 정답 문장과 변역기를 통한 문장이 완전히 동일하지는 않지만 어느정도 비슷한 부분을 볼 수가 있다.

이는 세밀한 전처리와 많은 데이터들을 이용하면 커버 가능할 것으로 보여진다. 

루브릭

아래의 기준을 바탕으로 프로젝트를 평가합니다.

평가문항	상세기준
1. 번역기 모델 학습에 필요한 텍스트 데이터 전처리가 잘 이루어졌다.

구두점, 대소문자, 띄어쓰기 등 번역기 모델에 요구되는 전처리가 정상적으로 진행되었다.

2. seq2seq 기반의 번역기 모델이 정상적으로 구동된다.

정상적으로 구동 되었다.

3. 테스트 결과 의미가 통하는 수준의 번역문이 생성되었다.

한글 번역까지는 하지 못하였지만 번역이 잘 된 모습이다.